### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
- mushroomの50サンプル分

### yのサンプリング
- 条件に合うまでサンプルを作り続ける

### 改訂(08/03)
- ori_tiのkeyがなかった場合に1とする必要がない
- sum([j[1] for j in ti_list])は必ず100である
- 正確な分布を作ることが必要なので、yを全列挙する必要がある。

In [1]:
#!pip3 install dwave-neal

In [2]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [4]:
num_samples

16

In [5]:
num_cols

119

In [6]:
y_sum

3

### t^0のみ

In [7]:
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum == 0:
        y_list.append(y)
len(y_list)

560

In [9]:
#fig, ax = plt.subplots(int(num_cols/2), 2, figsize=(100,200))

minP_cols = []
servey_num_col_list = columns

enul_p = {}
for index, i in enumerate(servey_num_col_list):
    
    ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

    ti_dic = {} #{内積の値:該当数}という辞書。

    for y in y_list:       
        if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
            ti_dic[int(np.dot(X_mushroom[i], y))] += 1
        else:
            ti_dic[int(np.dot(X_mushroom[i], y))] = 1

    ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
    franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
    enul_p[i] = franc/len(y_list)
    
    n_row = int(index%int(num_cols/2))
    
    if index < int(num_cols/2):
        n_col = 0
    else:
        n_col = 1
    
    '''#ヒストグラムを出力
    ax[n_row, n_col].set_title(i, fontsize=15)
    ax[n_row, n_col].bar([j[0] for j in ti_list], [j[1] for j in ti_list])
    ax[n_row, n_col].scatter(ori_ti, 100, s=200, marker='D')
    ax[n_row, n_col].set_xlabel('inner product size', fontsize=10)
    ax[n_row, n_col].set_ylabel('number', rotation=90, fontsize=10, labelpad=20)
    
#図が重ならないようにする
fig.tight_layout()

fig.subplots_adjust(bottom=0, left=0, top=0.1, right=0.1, wspace=0)
    
#全体に一つの題名を表示
fig.suptitle('inner product size and its number use all y which meet conditions', fontsize=15)

#図を表示
plt.show()

fig.savefig('../output/mushrooml/enul_p.png')'''

enul_p = pd.DataFrame.from_dict(enul_p, orient='index').T
print(enul_p)

enul_p.to_csv('../output/mushrooml/enul_p.csv')

     0         1         2    3    4         5     6    7    8         9  ...  \
0  1.0  0.001786  0.785714  1.0  1.0  0.607143  0.35  1.0  0.9  0.785714  ...   

   109  110  111       112   113  114  115  116  117     118  
0  1.0  0.9  1.0  0.785714  0.35  1.0  1.0  1.0  1.0  0.1875  

[1 rows x 119 columns]


In [6]:
def make_y(minP_cols):
    y_list = []
    y_list_all = itertools.product('01', repeat=num_samples)

    for y_tuple in y_list_all:
        y_list_list = [int(n) for n in list(y_tuple)]
        y = pd.Series(y_list_list)

        if sum(y) - y_sum != 0:
            continue

        #print(y)
        if len(minP_cols) == 0:
            y_list.append(y)
        else:
            for i in minP_cols:
                if np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom) != 0:
                    continue        
            y_list.append(y)
    return y_list

In [ ]:
minP_cols = []
servey_num_col_list = columns

while len(servey_num_col_list) > 0:    
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        y_list = make_y(minP_cols)
        
        for y in y_list:       
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        count_better_t = sum([v for k, v in ti_dic.items() if k >= ori_ti])
        p[i] = count_better_t / len(y_list)
        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]

In [ ]:
minP_cols